In [70]:
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import calendar
from scipy import stats

In [71]:
path = (r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\PrecipComparison' + 
        r'\snowfall_comp')
os.chdir(path)

In [72]:
akr_csv = (r"C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\PrecipComparison" +
           r"\snowfall_comp\akr02_precip_snow_coopcomp.csv")
bnv_csv = (r"C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\PrecipComparison" +
           r"\snowfall_comp\bnv01_precip_snow_coopcomp.csv")
orm_csv = (r"C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\PrecipComparison" + 
           "\snowfall_comp\orm01_precip_snow_coopcomp.csv")
rfd_csv = (r"C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\PrecipComparison" + 
           r"\snowfall_comp\rfd01_precip_snow_coopcomp.csv")
wls_csv = (r"C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\PrecipComparison" + 
           r"\snowfall_comp\wls01_precip_snow_coopcomp.csv")

In [73]:
akr_df = pd.read_csv(akr_csv)
bnv_df = pd.read_csv(bnv_csv)
orm_df = pd.read_csv(orm_csv)
rfd_df = pd.read_csv(rfd_csv)
wls_df = pd.read_csv(wls_csv)

In [74]:
frames = [akr_df,bnv_df,orm_df,rfd_df,wls_df]
df = pd.concat(frames)

In [75]:
df = df.replace('M',np.nan,regex=True)
df_traces = df.replace('T',0.005,regex=True)
df_no_traces = df.replace('T',0,regex=True)

In [76]:
df_traces.coop_precip = df_traces.coop_precip.astype(float)
df_no_traces.coop_precip = df_no_traces.coop_precip.astype(float)

df_traces.snow = df_traces.snow.astype(float)
df_no_traces.snow = df_no_traces.snow.astype(float)

df_traces['snowfall'] = np.nan
df_no_traces['snowfall'] = np.nan

df_traces['snowfall'] = np.where(df_traces['snow'] > 0,1,df_traces['snowfall'])
df_no_traces['snowfall'] = np.where(df_no_traces['snow'] > 0,1,df_no_traces['snowfall'])



In [77]:
df_traces['date'] = pd.to_datetime(df_traces.date,errors='coerce')
df_no_traces['date'] = pd.to_datetime(df_no_traces.date,errors='coerce')

In [78]:
df_traces['month'] = df_traces.date.dt.month
df_no_traces['month'] = df_no_traces.date.dt.month

df_traces['year'] = df_traces.date.dt.year
df_no_traces['year'] = df_no_traces.date.dt.year

In [79]:
stuff = ['Unnamed: 0']
df_traces = df_traces.drop(stuff,axis=1)
df_no_traces = df_no_traces.drop(stuff,axis=1)

In [80]:
monthly_traces = df_traces.groupby(['month','year']).sum().reset_index()
monthly_no_traces = df_no_traces.groupby(['month','year']).sum().reset_index()


In [81]:
cols = ['month','year','coag_precip','coop_precip','snow','snowfall']

monthly_trace_pp = monthly_traces[cols]
monthly_no_trace_pp = monthly_no_traces[cols]

In [82]:
it_snowed = monthly_trace_pp.snow > 0
monthly_trace_pp = monthly_trace_pp[it_snowed]

it_snowed = monthly_no_trace_pp.snow > 0
monthly_no_trace_pp = monthly_no_trace_pp[it_snowed]

In [83]:
#monthly_trace_pp['month'] = monthly_trace_pp.month.astype(int)
#monthly_no_trace_pp['month'] = monthly_no_trace_pp.month.astype(int)

monthly_trace_pp['month'] = monthly_trace_pp['month'].apply(lambda x: calendar.month_abbr[x])
monthly_no_trace_pp['month'] = monthly_no_trace_pp['month'].apply(lambda x: calendar.month_abbr[x])

monthly_trace_pp['year'] = monthly_trace_pp.year.astype(str)
monthly_no_trace_pp['year'] = monthly_no_trace_pp.year.astype(str)

monthly_trace_pp['ym'] = (monthly_trace_pp['month']) + '-' + (monthly_trace_pp['year'])
monthly_no_trace_pp['ym'] = (monthly_no_trace_pp['month']) + '-' + (monthly_no_trace_pp['year'])

monthly_trace_pp['snowfall'] = monthly_trace_pp.snowfall.astype(str)
monthly_no_trace_pp['snowfall'] = monthly_no_trace_pp.snowfall.astype(str)

In [84]:
monthly_trace_pp['rez'] = monthly_trace_pp.coag_precip - monthly_trace_pp.coop_precip
monthly_no_trace_pp['rez'] = monthly_no_trace_pp.coag_precip - monthly_no_trace_pp.coop_precip

In [85]:
monthly_trace_pp = monthly_trace_pp.round(decimals=2)
monthly_no_trace_pp = monthly_no_trace_pp.round(decimals=2)

In [89]:
precip_comp_fig_trace = go.Figure(
    data=[go.Scatter()],
    layout=go.Layout(
        title=go.layout.Title(
            text='Monthly Preicp Difference vs Monthly Snowfall (Traces Included)'),
        xaxis=go.layout.XAxis(
            title=go.layout.xaxis.Title(text='Snowfall (inches)')),
        yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text='Precip Differnce (inches)'))))

precip_comp_fig_trace.add_trace(
    go.Scatter(x=monthly_trace_pp.snow,y=monthly_trace_pp.rez,
               text=monthly_trace_pp.ym + '<br \>' + 'Snowfall Events: ' + monthly_trace_pp.snowfall,
               mode='markers'))

In [90]:
precip_comp_fig_no_trace = go.Figure(
    data=[go.Scatter()],
    layout=go.Layout(
        title=go.layout.Title(
            text='Monthly Precip Difference vs Monthly Snowfall (No Traces)'),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(text='Snowfall (inches)')),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(text='Precip Difference (Inches)'))))

precip_comp_fig_no_trace.add_trace(
    go.Scatter(x=monthly_no_trace_pp.snowfall,y=monthly_no_trace_pp.rez,
              text=monthly_no_trace_pp.ym + '<br \>' +
              'Snowfall Events: ' + monthly_no_trace_pp.snowfall,
              mode='markers'))